In [0]:
%fs ls dbfs:/public/

path,name,size,modificationTime
dbfs:/public/retail_db/,retail_db/,0,1736895018908


In [0]:
%fs ls dbfs:/public/retail_db

path,name,size,modificationTime
dbfs:/public/retail_db/README.md,README.md,806,1736895021611
dbfs:/public/retail_db/categories/,categories/,0,1736895024588
dbfs:/public/retail_db/create_db.sql,create_db.sql,10303297,1736895023301
dbfs:/public/retail_db/create_db_tables_pg.sql,create_db_tables_pg.sql,1748,1736895020760
dbfs:/public/retail_db/customers/,customers/,0,1736895019295
dbfs:/public/retail_db/departments/,departments/,0,1736895021010
dbfs:/public/retail_db/load_db_tables_pg.sql,load_db_tables_pg.sql,10297372,1736895026022
dbfs:/public/retail_db/order_items/,order_items/,0,1736895021950
dbfs:/public/retail_db/orders/,orders/,0,1736895023875
dbfs:/public/retail_db/products/,products/,0,1736895020073


In [0]:
%sql

select *
from TEXT.`/public/retail_db/orders/`
limit 10

value
"1,2013-07-25 00:00:00.0,11599,CLOSED"
"2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT"
"3,2013-07-25 00:00:00.0,12111,COMPLETE"
"4,2013-07-25 00:00:00.0,8827,CLOSED"
"5,2013-07-25 00:00:00.0,11318,COMPLETE"
"6,2013-07-25 00:00:00.0,7130,COMPLETE"
"7,2013-07-25 00:00:00.0,4530,COMPLETE"
"8,2013-07-25 00:00:00.0,2911,PROCESSING"
"9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT"
"10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT"


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW orders (
  order_id INT,
  order_date DATE,
  order_customer_id INT,
  order_status STRING
)  USING csv 
OPTIONS (
  path='/public/retail_db/orders',
  sep=','
)
   

In [0]:
%sql
DESCRIBE orders

col_name,data_type,comment
order_id,int,null
order_date,date,null
order_customer_id,int,null
order_status,string,null


In [0]:
%sql
select *
from TEXT.`/public/retail_db/order_items/`
limit 10

value
"1,1,957,1,299.98,299.98"
"2,2,1073,1,199.99,199.99"
"3,2,502,5,250.0,50.0"
"4,2,403,1,129.99,129.99"
"5,4,897,2,49.98,24.99"
"6,4,365,5,299.95,59.99"
"7,4,502,3,150.0,50.0"
"8,4,1014,4,199.92,49.98"
"9,5,957,1,299.98,299.98"
"10,5,365,5,299.95,59.99"


In [0]:

%sql
CREATE OR REPLACE TEMPORARY VIEW order_items (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
)  USING csv 
OPTIONS (
  path='/public/retail_db/order_items',
  sep=','
)
    

In [0]:
%sql
DESCRIBE order_items

col_name,data_type,comment
order_item_id,int,null
order_item_order_id,int,null
order_item_product_id,int,null
order_item_quantity,int,null
order_item_subtotal,float,null
order_item_product_price,float,null


In [0]:
%sql
select *
from orders
limit 10

order_id,order_date,order_customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [0]:
%sql
select *
from order_items
limit 10

order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99


In [0]:
%sql
INSERT OVERWRITE DIRECTORY 'dbfs:/public/retail_db/daily_product_revenue' 
USING PARQUET

select 
  order_date,
  round(sum(order_item_quantity * order_item_product_price),2) as total
from orders o
left join order_items oi 
on o.order_id = oi.order_item_order_id
where order_status in ('COMPLETE',' CLOSED')
group by 1
HAVING round(sum(order_item_quantity * order_item_product_price),2) IS NOT NULL
ORDER BY 1
limit 50

In [0]:
%fs ls dbfs:/public/retail_db/daily_product_revenue

path,name,size,modificationTime
dbfs:/public/retail_db/daily_product_revenue/_SUCCESS,_SUCCESS,0,1736895224898
dbfs:/public/retail_db/daily_product_revenue/_committed_426740893539489616,_committed_426740893539489616,121,1736895224508
dbfs:/public/retail_db/daily_product_revenue/_started_426740893539489616,_started_426740893539489616,0,1736895220566
dbfs:/public/retail_db/daily_product_revenue/part-00000-tid-426740893539489616-2cdb16c2-5b2e-4004-8001-7ddbd031a3f1-7-1-c000.snappy.parquet,part-00000-tid-426740893539489616-2cdb16c2-5b2e-4004-8001-7ddbd031a3f1-7-1-c000.snappy.parquet,1300,1736895224193


In [0]:
%sql

select *
from parquet.`/public/retail_db/daily_product_revenue/`
limit 25

order_date,total
2013-07-25,20030.32
2013-07-26,42165.88
2013-07-27,33156.21
2013-07-28,27012.91
2013-07-29,45898.65
2013-07-30,40590.21
2013-07-31,46503.83
2013-08-01,38231.41
2013-08-02,36633.44
2013-08-03,34828.71
